In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# set pandas to display all columns
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('new_222_data_norm_decorr.csv')
df.head()

,prev_year_access_to_exercise_opportunities,prev_year_adult_obesity,prev_year_air_pollution_particulate_matter,prev_year_alcohol_impaired_driving_deaths,prev_year_children_in_single_parent_households,prev_year_dentists,prev_year_diabetes_monitoring,prev_year_drinking_water_violations,prev_year_driving_alone_to_work,prev_year_flu_vaccinations,prev_year_income_inequality,prev_year_injury_deaths,prev_year_long_commute_driving_alone,prev_year_low_birthweight,prev_year_mental_health_providers,prev_year_preventable_hospital_stays,prev_year_primary_care_physicians,prev_year_sexually_transmitted_infections,prev_year_social_associations,prev_year_unemployment,prev_year_uninsured,prev_year_violent_crime,coastline_flag,Top_Marginal_Personal_Income_Tax_Rate,Top_Marginal_Corporate_Income_Tax_Rate,"Personal_Income_Tax_Progressivity_(change in tax liability per $1,000 of income)","Property_Tax_Burden_(per $1,000 of personal income)","Sales_Tax_Burden(per $1,000 of personal income)","Remaining_Tax_Burden(per $1,000 of personal income)",Estate_Inheritance_Tax_Levied_Flag,Recently_Legislated_Tax_Changes,"Public_Employees_Per_10,000_of_Population_(full-time equivalent)","State_Liability_System_Survey_(tort litigation treatment, judicial impartiality, etc.)",State_Minimum_Wage_(federal floor is $7.25),Average Workers_Compensation_Costs_(per $100 of payroll),Right-to-Work_State_Flag_(option to join or support a union),Tax_Expenditure_Limits,loan,office,Business plan templates,Small business ideas,website hosting service,tax_pct,loan_pct,business_pct,rent_pct,college_pct,apartments_pct,office_pct,insurance_pct,Small business ideas_pct,office space for rent_pct,website hosting service_pct,freelancing_pct,side hustle_pct,ecommerce_pct,death_pct_diff,GPT_Autoencoder_dim3,GPT_PCA_dim0,GPT_PCA_dim1,GPT_PCA_dim2,GPT_PCA_dim3,GPT_PCA_dim4,GPT_PCA_dim5,GPT_PCA_dim6,GPT_PCA_dim7,GPT_PCA_dim8,GPT_PCA_dim9,GPT_OH_Connected_Economy,GPT_OH_Government_Support,GPT_OH_Large_Customer_Base,GPT_OH_Low_Cost_of_Living,GPT_OH_Natural_Resources,GPT_OH_Supportive_Community,GPT_OH_Limited_Financing,GPT_OH_Limited_Tech_n_Infra,GPT_OH_Low_Pop_Bad_Weather,GPT_OH_Tough_Labor_Market,GPT_OH_Unfavorable_Location,GPT_OH_Various_Downsides,pop_5yr_pct_chg,pop_10yr_pct_chg,Prev_Yr_Agri_Pct_GDP,Prev_Yr_Mine_Pct_GDP,Prev_Yr_Retl_Pct_GDP,Prev_Yr_Manu_Pct_GDP,Prev_Yr_Info_Pct_GDP,Prev_Yr_FinS_Pct_GDP,Prev_Yr_EduS_Pct_GDP,Prev_Yr_EntS_Pct_GDP,Prev_Yr_Gvmt_Pct_GDP,Prev_Yr_GoMn_Pct_GDP,Prev_Yr_GoTr_Pct_GDP,Prev_Yr_GoMf_Pct_GDP,two_yrs_prior_pct_it_workers,finance_planning_pca_0,finance_planning_pca_1,finance_planning_pca_2,finance_planning_pca_3,health_social_pca_0,health_social_pca_1,health_social_pca_2,health_social_pca_3,population_2020,gpt_ae_pca_0,gpt_ae_pca_1,gpt_ae_pca_2,microbusiness_density
0,0.255810,-0.086091,1.649137,0.362961,0.059623,-0.004225,-0.110740,-0.804684,0.849884,-0.119032,0.780584,-0.822201,0.479138,0.402184,0.807730,1.134093,-0.251409,0.374158,0.102851,-0.902340,-0.270715,0.130270,-0.296240,-0.390919,-0.219304,-1.212214,-1.868572,0.280925,0.937401,-0.590955,0.493912,0.650154,-0.679811,-0.815306,-0.074044,0.661686,-0.974352,0.07454,2.53845,-0.107763,0.000391,-0.198629,-0.9935,-0.885835,-1.248157,-0.176387,0.837903,0.080635,0.148224,-0.535565,-1.543214,2.906663,-0.015146,-0.862067,2.682848,-0.003526,0.269209,2.001423,1.529495,0.451515,-0.315582,1.259285,0.371456,-0.117666,-1.197495,-0.084728,-0.941355,-0.558370,1.954957,-1.527862,-1.214838,0.212098,-0.447399,-0.595553,-1.514046,-0.905118,-0.664757,-0.898142,1.568273,-0.655792,0.324732,0.153961,-0.524757,-0.234408,0.194062,0.113404,0.621425,-0.066865,-0.037589,-0.020735,-0.344793,-0.485444,2.549471,0.322877,-0.717311,0.701179,-1.024085,-1.306739,0.785928,0.241566,0.801845,0.006789,-0.525766,-0.146661,4.886915,-2.578624,-0.587025,3.376788
1,0.472213,-0.593031,-0.257468,0.161051,-0.242573,-0.388414,-0.269831,-0.804684,0.578838,0.284172,-0.137172,-0.636171,0.639550,-0.085835,-0.306999,-0.616952,-0.561000,-0.391130,-0.18681

In [3]:
# create a copy of the dataframe
df_copy = df.copy()
X = df_copy.iloc[:, :-1]
y = df_copy.iloc[:, -1]

X_norm = X.copy()
y = y.copy()

In [4]:
from sklearn.feature_selection import SequentialFeatureSelector
N_features_sel = 20

In [5]:
# forward selection with linear regression
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
sfs = SequentialFeatureSelector(lr_model, n_features_to_select=N_features_sel, direction='forward', n_jobs=-1)
sfs.fit(X_norm, y)
lr_forward_importants = sfs.get_feature_names_out()

In [6]:
X_lr = X_norm[lr_forward_importants]
y_lr = y 

# use statsmodels to get the p-values
import statsmodels.api as sm
X_lr = sm.add_constant(X_lr)
lr_model = sm.OLS(y_lr, X_lr).fit()
# rank the features by their p-values
lr_p_values = lr_model.pvalues
lr_p_values = lr_p_values.sort_values(ascending=True)

# sorted features by p-values as a list
lr_sorted = list(lr_p_values.index)

In [7]:
# forward selection with KNN regression
from sklearn.neighbors import KNeighborsRegressor

knn_model = KNeighborsRegressor()
sfs = SequentialFeatureSelector(knn_model, n_features_to_select=N_features_sel, direction='forward', n_jobs=-1)

sfs.fit(X_norm, y)
knn_forward_importants = sfs.get_feature_names_out()

In [8]:
# forward selection with DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

dt_model = DecisionTreeRegressor()
sfs = SequentialFeatureSelector(dt_model, n_features_to_select=N_features_sel, direction='forward', n_jobs=-1)
sfs.fit(X_norm, y)
dt_forward_importants = sfs.get_feature_names_out()

In [9]:
X_dt = X_norm[dt_forward_importants]
y_lt = y

# fit a decision tree model and get the feature importances
dt_model = DecisionTreeRegressor()
dt_model.fit(X_dt, y_lt)
dt_importances = dt_model.feature_importances_
dt_importances = pd.Series(dt_importances, index=X_dt.columns)
dt_importances = dt_importances.sort_values(ascending=False)
dt_sorted = list(dt_importances.index)

In [10]:
# forward selection with Random Forest
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(max_depth=3, n_estimators=30)
sfs = SequentialFeatureSelector(rf_model, n_features_to_select=N_features_sel, direction='forward', n_jobs=-1)
sfs.fit(X_norm, y)
rf_forward_importants = sfs.get_feature_names_out()

In [11]:
# fit a random forest model and get the feature importances
X_rf = X_norm[rf_forward_importants]
y_rf = y
rf_model = RandomForestRegressor(max_depth=3, n_estimators=30)
rf_model.fit(X_rf, y_rf)
rf_importances = rf_model.feature_importances_
rf_importances = pd.Series(rf_importances, index=X_rf.columns)
rf_importances = rf_importances.sort_values(ascending=False)
rf_sorted = list(rf_importances.index)

In [12]:
# forward selection with BayesianRidge
from sklearn.linear_model import BayesianRidge

br_model = BayesianRidge()
sfs = SequentialFeatureSelector(br_model, n_features_to_select=N_features_sel, direction='forward', n_jobs=-1)
sfs.fit(X_norm, y)
br_forward_importants = sfs.get_feature_names_out()

In [13]:
X_br = X_norm[br_forward_importants]
y_br = y

br_model = BayesianRidge()
br_model.fit(X_br, y_br)
br_importances = br_model.coef_
br_importances = pd.Series(br_importances, index=X_br.columns)
br_importances = br_importances.sort_values(ascending=False)
br_sorted = list(br_importances.index)

In [14]:
# put all the forward selection results into a dataframe
forward_selection_df = pd.DataFrame({'OLS': lr_sorted[:-1],
                                        'KNN': knn_forward_importants,
                                        'DecisionTree': dt_sorted,
                                        'RandomForest': rf_sorted,
                                        'BayesianRidge': br_sorted})
forward_selection_df

,OLS,KNN,DecisionTree,RandomForest,BayesianRidge
0,const,prev_year_dentists,GPT_PCA_dim4,prev_year_access_to_exercise_opportunities,health_social_pca_2
1,health_social_pca_0,prev_year_low_birthweight,prev_year_access_to_exercise_opportunities,pop_10yr_pct_chg,Prev_Yr_FinS_Pct_GDP
2,health_social_pca_2,prev_year_mental_health_providers,Prev_Yr_GoTr_Pct_GDP,two_yrs_prior_pct_it_workers,pop_5yr_pct_chg
3,pop_5yr_pct_chg,prev_year_primary_care_physicians,health_social_pca_2,GPT_PCA_dim3,health_social_pca_0
4,"Remaining_Tax_Burden(per $1,000 of personal in...",prev_year_sexually_transmitted_infections,GPT_OH_Supportive_Community,prev_year_adult_obesity,"Remaining_Tax_Burden(per $1,000 of personal in..."
5,Prev_Yr_FinS_Pct_GDP,Business plan templates,State_Minimum_Wage_(federal floor is $7.25),population_2020,population_2020
6,population_2020,side hustle_pct,GPT_OH_Limited_Tech_n_Infra,Prev_Yr_EntS_Pct_GDP,Prev_Yr_EntS_Pct_GDP
7,health_social_pca_1,death_pct_diff,Business plan templates,Prev_Yr_FinS_Pct_GDP,health_social_pca_1
8,prev_year_long_commute_driving_alone,GPT_PCA_dim2,coastline_flag,Prev_Yr_Manu_Pct_GDP,prev_year_children_in_single_parent_households
9,prev_year_adult_obesity,GPT_PCA_dim5,GPT_OH_Limited_Financing,prev_year_unemployment,website hosting service_pct


In [15]:
# save the forward selection results to a csv file
forward_selection_df.to_csv('forward_selection_result_decorr.csv', index=False)